Lets import the necessary libraries to help in this analysis

In [22]:
import os
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

We need to import the data, but first there are several files that we need to work. The data is separated in terms of decade. Therefore, we write a script that looks for all the files containing the data, read and combine them together into a single dataframe that we can work on.

In [4]:
# Get the current working directory
cwd = os.getcwd()

In [9]:
files = os.listdir(cwd)

dfs = []

for file in files:
    
    if file.endswith('.csv'):
        
        df = pd.read_csv(file)
        dfs.append(df)
        
df = pd.concat(dfs, ignore_index = True)
df.shape

(667, 14)

### Data overview 

Let us have an overview of the dataset

In [10]:
df.head(4)

,title,artist,top_genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,No Scrubs,TLC,atl hip hop,1999,93,68,74,-4,7,59,214,3,10,79
1,I Want It That Way,Backstreet Boys,boy band,2001,99,70,69,-6,16,48,214,22,3,76
2,It Wasn't Me,Shaggy,dance pop,2000,95,61,85,-5,31,65,228,6,7,76
3,"Because You Loved Me (Theme from ""Up Close and...",CÃ©line Dion,canadian pop,1996,120,47,61,-10,10,18,274,31,3,74


In [11]:
df.columns

Index(['title', 'artist', 'top_genre', 'year', 'bpm', 'nrgy', 'dnce', 'dB',
       'live', 'val', 'dur', 'acous', 'spch', 'pop'],
      dtype='object')

1. top genre: genre of the song

2. year: year of the song (due to re-releases, the year might not correspond to the release year of the original song)

3. bpm(beats per minute): beats per minute

4. nrgy(energy): energy of a song, the higher the value the more energetic the song is

5. dnce(danceability): the higher the value, the easier it is to dance to this song.

6. dB(loudness): the higher the value, the louder the song.

7. live(liveness): the higher the value, the more likely the song is a live recording.

8. val(valence): the higher the value, the more positive mood for the song.

9. dur(duration): the duration of the song.

10. acous(acousticness): the higher the value the more acoustic the song is.

11. spch(speechiness): the higher the value the more spoken word the song contains.

12. pop(popularity): the higher the value the more popular the song is.

##### Business Question 

Now let us formulate a business question for this project

1. What was the most popular genre in each decade?
2. Who was the most listened to artist in each decade?
2. What was the average danceability score of most recommended songs in every decade? 
3. What was the average bpm score of most recommended songs in every decade? 
4. What is the secret behind a popular song on online streaming platforms?

### Data Cleaning 

Let us drop unnecessary columns

In [12]:
df = df.drop(['year', 'title'], axis = 1)
df.head(4)

,artist,top_genre,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,TLC,atl hip hop,93,68,74,-4,7,59,214,3,10,79
1,Backstreet Boys,boy band,99,70,69,-6,16,48,214,22,3,76
2,Shaggy,dance pop,95,61,85,-5,31,65,228,6,7,76
3,CÃ©line Dion,canadian pop,120,47,61,-10,10,18,274,31,3,74


Let us check for missing datapoints

In [13]:
attributes = ['artist', 'top_genre', 'bpm', 'nrgy', 'dnce', 'dB',
       'live', 'val', 'dur', 'acous', 'spch', 'pop']

In [ ]:
for attribute in attributes:
    
    null_check = pd.isnull(df[attribute])
    print(df[null_check])

We can see that the 'top_genre' column has several missing values. We can drop them

<div class="alert alert-block alert-warning">

<b>!! WARNING !!</b> This is a small dataset and dropping some can hugely compromise the integrity of the analytical results.

</div>

So we are gping to use a different technique to fill the missing values. We fill any missing values with the most popular value in the corresponding column

Let us iterate through the columns, find the common value and then fill it where there is any missing value.

In [15]:
for column in df.columns:
    
    mode_value = df[column].mode()[0]
    df[column].fillna(mode_value, inplace = True)

Let us confirm that all the missing values have been replaced

In [16]:
attributes = ['top_genre']

In [17]:
for attribute in attributes:
    
    null_check = pd.isnull(df[attribute])
    print(df[null_check])

Empty DataFrame
Columns: [artist, top_genre, bpm, nrgy, dnce, dB, live, val, dur, acous, spch, pop]
Index: []


We do not have any missing values hence the loop worked successfully. So let us answer the business questions.

###### 1. What was the most popular genre in each decade?

In [23]:
dfs = []
years = ['1950', '1960', '1970', '1980', '1990', '2000', '2010']

for file in files:
    
    if file.endswith('.csv'):
        
        df = pd.read_csv(file)
        df['decade'] = ''
        
        for i in range(len(df)):
            
            for year in years:
                
                if year in file:
                    
                    df['decade'].loc[i] = year
                    
        dfs.append(df)
df = pd.concat(dfs, ignore_index = True)
df.head(4)

,title,artist,top_genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,decade
0,No Scrubs,TLC,atl hip hop,1999,93,68,74,-4,7,59,214,3,10,79,1990
1,I Want It That Way,Backstreet Boys,boy band,2001,99,70,69,-6,16,48,214,22,3,76,1990
2,It Wasn't Me,Shaggy,dance pop,2000,95,61,85,-5,31,65,228,6,7,76,1990
3,"Because You Loved Me (Theme from ""Up Close and...",CÃ©line Dion,canadian pop,1996,120,47,61,-10,10,18,274,31,3,74,1990


Let us confirm if the decade is an integer or a string using the following loop.

In [ ]:
for i in range(len(df)):
    
    print(type(df['decade'].loc[i]))

Let us convert the string values to integers using the code below

In [25]:
df['decade'] = df['decade'].astype(int)

Let us confirm if the decades are now integers

In [ ]:
for i in range(len(df)):
    
    print(type(df['decade'].loc[i]))

Let us check the available genres in our dataset

In [ ]:
df['top_genre'].unique()

In [30]:
df.groupby('decade').top_genre.agg(pd.Series.mode).reset_index(name = 'Genre')

,decade,Genre
0,1950,adult standards
1,1960,adult standards
2,1970,album rock
3,1980,album rock
4,1990,dance pop
5,2000,dance pop
6,2010,dance pop


##### 2. Who was the most listened to artist in each decade? 

In [31]:
df.groupby('decade').artist.agg(pd.Series.mode).reset_index(name = 'Artist')

,decade,Artist
0,1950,"[Craig Douglas, Dean Martin, The Fleetwoods]"
1,1960,The Beatles
2,1970,"[Elton John, Queen]"
3,1980,Madonna
4,1990,Mariah Carey
5,2000,Rihanna
6,2010,Post Malone


##### 3. What was the average danceability score of most recommended songs in every decade? 

In [32]:
df.groupby('decade').dnce.agg(['count', 'mean'])

,count,mean
decade,,
1950,73,51.027397
1960,97,51.175258
1970,104,56.596154
1980,105,62.276190
1990,88,65.147727
2000,100,68.530000
2010,100,65.370000
